<div>
<img src='assets\VFM_logo.gif' alt="Drawing" style="width: 400px; float: right;"/>
</div>

# Controleren marktdata 

- Dagelijkse controle van marktdata op afwijkingen.
<br>
<br>
  **Merijn van Miltenburg, 2020


# Inhoudsopgave

- [Inleiding](#inleiding)
- [Inlezen van de rates](#inlezen)
- [Controleren op Null Waarden](#controleren)
- [Expand Time series](#expand)
- [Berekenen van Outliers](#outliers)
- [Toevoegen Dimensions](#dimensions)
- [Definieer uitzonderingen](#uitzonderingen)
- [Presentatie van resultaten (Plotly - Dash)](#presentatie)

<a id='inleiding'></a>
# Inleiding

Dit is een bestaande case binnen de Volksbank Financial Markets. Data kwaliteit is van belang voor financiale en risicorapportages. Wij lezen dagelijks rond de 10.000 datapunten in. Door de data te visualeren en afwijkingen er uit te lichten proberen we de data kwaliteit direct bij het inlezen van de data te verbeteren. In een volgende stap willen we ook machine learning gaan toepassen om mogelijke anomalies in de data beter naar voren te halen.

- De dataset kan worden gesplits in verschillende curve types:
    * Yield heeft betrekking op rente curves. Dit zijn rente curves die worden gebruikt voor de waardering van de derivaten (afgeleide financiele instrumenten).     
    * Capital Price zijn prijzen van beursgenoteerde obligaties.     
    * Futures betreft marktdata van termijn contracten. 
    * Points slaan op de FRA punten (Forward Rate Agreements). Dit zijn niet beursgenoteerde rente termijncontracten. 
    * FX staat voor Foreign Exchange. Dit zijn wisselkoersen voor vreemde valuta. 
<br><br>
- Dit zijn niet alle type marktdata die we dagelijks inlezen. O.a. Rentegevoeligheden (volatilities) zijn in deze presentatie niet meegenomen.

In [1]:
import pandas as pd                                     # 1.05
import numpy as np                                      # 1.18

# berekenen datum actual_dt
from pandas.tseries.offsets import DateOffset           
import os

# Voor inlezen dimensies
import json             

# Voor inlezen ECB Close Rates
import requests                                         # 2.24
import io                                               

#Voor printen dateframe
from IPython.core.display import HTML                   # 7.16

# Voor meten doorlooptijd van de Python code
import time  

# Voor jupyter dash dashboard
import plotly.express as px                             # Plotly Express
import plotly.graph_objects as go                       # Plotly Graphic objects
import dash_core_components as dcc                      # Dash Core Components
import dash_html_components as html                     # Dash html components
import dash_table as dct                                # Dash Data table
from jupyter_dash import JupyterDash                    # run Dash in Jupyter
from dash.dependencies import Input, Output, State      # voor callbacks
from dash import callback_context                       # Voor callback info205100
                                           

<a id='inlezen'></a>
# Inlezen van de rates 


In [2]:

def read_data():
    """
    Lees de te controleren marktdata in.
    
    In productie wordt dit via SQL uit het bron systeem gelezen.    
    Voor de demo heb ik de data eerst gedownload naar CSV en lezen we hier de CSV in.
    Deze data is gemanipuleerd. De oorspronkelijke data is helaas niet publiek toegankelijk.
    De data is alleen bedoelt voor demo doeleinden.
        
    Input: 
      -        
    Output: 
      Pandas Dataframe
      
    """          
    try:
    
        ROOT_DIR = os.path.dirname(os.path.abspath("LICENCE.TXT"))
        
        # Datum kolommen
        date_cols = [
            'rate_dt', 'input_dt', 'actual_dt', 'start_term_dt', 'maturity_term_dt'                                        
            ]
    
        # Omzetten datum kolommen. int_basis en exchange name zijn gewoon string, 
        # maar werden niet goed herkend
        df = pd.read_csv(ROOT_DIR + r'\data\rates_data.csv', 
                         parse_dates = date_cols, 
                         dtype ={"int_basis":'S10', "exchange_name": 'S10'} )

        # Verwijder kolommen die we niet nodig hebben
        df = df.drop(
            [
                'seq_no', 
                'date_days',
                'int_basis',
                'int_days',
                'start_term_dt',
                'maturity_term_dt',
                'start_term_date_days',
                'maturity_term',
                'maturity_term_date_days',
                'exchange_name',
                'bid',
                'offer',
                'days_off'
            ],
            axis=1
        )

    except Exception as e:
            print("ERROR: Unable to find or access file:", e)
    
    return df

def show(df):
    """ Toon een pandas dataframe in HTML."""
    if type (df) == pd.core.frame.DataFrame:
        display(HTML(df.to_html(index=False))) 
    else:
        print("input is not a dataframe. Type not recognized " + str(type(df)))        
                

# Inlezen data
df_rates = read_data()    

show(df_rates.head())


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid
2020-01-01,European CPI,EUR,Fixing,Yield,2020-02-24,1 MONTH,2020-02-03,NaN,104.03246
2020-01-01,Netherlands CPI Base 2015,EUR,Fixing,Yield,2020-02-24,1 MONTH,2020-02-03,NaN,105.93230
2020-01-01,Spain CPI Base 2015,EUR,Fixing,Yield,2020-02-24,1 MONTH,2020-02-03,NaN,103.38178
2020-01-02,BBSW,AUD,Fixing,Yield,2020-01-02,1 MONTH,2020-02-03,NaN,0.84834
2020-01-02,BBSW,AUD,Fixing,Yield,2020-01-02,2 MONTHS,2020-03-02,NaN,0.87792


# Inlezen ECB Close Rates

Ik lees van de ECB site de daily close FX rates in, om deze later met onze FX rates te vergelijken
hierbij krijgen we grotere afwijkingen dan in productie omdat ik de demo data heb aangepast.

In [3]:

def ecb_close_rates(df):    
    """
    Laad ECB Close Rates in een dataframe.
    
    ECB Close rates worden geladen vanaf de site van de ECB.    
    Deze data wordt vervolgens omgezet van EUR gebaseerd (dus bv EUR/CHF) naar USD (dus USD/CHF).     
    """   
           
    start_dt = df_rates.rate_dt.min().strftime("%Y-%m-%d")
    end_dt = df_rates.rate_dt.max().strftime("%Y-%m-%d")        
        
    # Opbouwen van de URL
    
    entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/'  # Using protocol 'https'
    resource = 'data'           # The resource for data queries is always'data'    
    flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
    ccylist = ['USD', 'AUD', 'HUF', 
               'DKK', 'CAD', 'CHF', 
               'CZK', 'GBP', 'HKD',
               'JPY', 'NOK', 'NZD', 
               'PLN', 'SEK', 'SGD', 
               'TRY', 'ZAR']

    # Define the parameters
    parameters = {
        'startPeriod': start_dt,  # Start date of the time series
        'endPeriod': end_dt     # End of the time series
    }

    
    column_names = ["ccy", "rate_dt", "mid"]
    df = pd.DataFrame(columns = column_names)

    for ccy in ccylist:

        # Samenstellen URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
        key = 'D.' + ccy  + '.EUR.SP00.A'
        request_url = entrypoint + resource + '/'+ flowRef + '/' + key

        # Make the HTTP request            
        response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})

        # Check if the response returns succesfully with response code 200    
        if response.status_code == 200:   
            # Read the response as a file into a Pandas DataFrame
            dfi = pd.read_csv(io.StringIO(response.text))
            dft = dfi[['CURRENCY','TIME_PERIOD', 'OBS_VALUE']]        
            dft.columns = column_names        
            df = df.append(dft,ignore_index=True)
        else:
            print ('Error retrieving data for ccy ' + ccy + ' ' + str(response.status_code)  )

    return df
        
# Vertalen EUR rates naar USD rates
def get_usd_rate(df, ccy, date):
    """
    Translate EUR rate to USD rate
    Input:
        DF   : Pandas Dataframe
        ccy  : string
        date : date
    Output
       rate  : USD rate for this ccy and date    
    """    
    if ccy == 'USD':
        # EUR ccy is dominant (EUR/USD) - dus hier hoeft niets vertaald te worden
        return df[(df['ccy']== ccy) & (df['rate_dt']== date)]['mid'].values[0]   
    
    else:
        
        # Vertaal EUR/ccy naar USD/ccy
        # EUR/USD
        eurusd_rate = df[(df['ccy']=='USD') & (df['rate_dt']==date)]['mid'].values[0]  
        
        # EUR/fcy
        fcy_rate = df[(df['ccy']== ccy) & (df['rate_dt']== date)]['mid'].values[0]
        
        # Gemenebest landen zijn dominant currencies (GBP/USD, AUD/USD, NZD/USD)
        if ccy in ['GBP','AUD','NZD']:
            rate = eurusd_rate / fcy_rate
        else:
            rate = fcy_rate / eurusd_rate        
        return  rate

# Haal ECB Closing Rates op
df_ecb = ecb_close_rates(df_rates)

# Pas deze functie toe om EUR rates om te rekenen naar USD rates
df_ecb['usdrate'] = df_ecb.apply(lambda x: get_usd_rate(df_ecb, x['ccy'],x['rate_dt']),axis=1).round(5)

# Vertaal ccy USD voor EUR omdat we nu USD rates hebben
df_ecb["ccy"].replace({"USD": "EUR"}, inplace=True)

# toon laatste data
print ('ECB Rates - inclusief USD rates')
show(df_ecb[df_ecb['rate_dt']==df_ecb['rate_dt'].max()].head())

ECB Rates - inclusief USD rates


ccy,rate_dt,mid,usdrate
EUR,2020-12-30,1.2281,1.22810
AUD,2020-12-30,1.6025,0.76637
HUF,2020-12-30,364.8800,297.10936
DKK,2020-12-30,7.4393,6.05757
CAD,2020-12-30,1.5701,1.27848


<a id='controleren'></a>
# Controleren en aanvullen Null Waarden

Eerste stap is de controle van de brongegevens. 

Bij Bonds en Bond Yields is de kolom actual_dt (value date van het datapunt) vaak niet ingevuld. Deze vul ik aan.
De FRA punten kennen geen waarde voor de actual_dt en een verkeerde waarde voor time_band. Deze bereken ik op basis van de rate date en de start term kolom en vul ik aan.

In [4]:
def clean_data(df):
    """
    Opschonen van de dataframe
    Input:
        df    Pandas data frame
    Output
        df    Pandas data frame    
    """
    
    # Module wordt getimed - omdat dit eerst met een loop erg traag werkte
    # gaat nu een stuk sneller
    t0 = time.time()
    
    # kopieer de dataframe die we willen opschonen
    df = df.copy()
    
    # actual date vullen op basis van Rate Date
    df['actual_dt'].fillna(df['rate_dt'], inplace= True)

    # Vertaal start_term en rate date naar actual_dt
    def get_fra_date(rate_dt, start_term):
        """
        Bereken de Actual Date voor FRA's. 
        De actual date wordt berekend door de rate_dt 
        + het aantal maanden in start_term.
        Input:
            rate_dt
            start_term
        """                
        return (rate_dt + DateOffset(months=start_term))        

    # Vertaal start_term naar time_band
    def get_timeband(start_term):
        """
        Deze functie geeft de tekstuele weergave van het aantal maanden.
        Input: 
            start_term: Numeriek
        Output:
            time_band: string        
        """        
        if start_term == 1:
            time_band = '1 MONTH'
        else:
            time_band = str(int(start_term)) + ' MONTHS'      
        return time_band        

    # Toepassen van vertaal functies
    # Voor elke FRA rate berekenen we de actual_dt en time_band
    mask = df['curve_type']=='Points'
    df.loc[mask,'actual_dt'] = df.loc[mask].apply(lambda x: get_fra_date(x['rate_dt'], x['start_term'])
                                              if x['curve_type']=='Points' else x['actual_dt']
                                                  , axis = 1)
    df.loc[mask,'time_band'] = df.loc[mask].apply(lambda x: get_timeband(x['start_term']) 
                                              if x['curve_type']=='Points' else x['time_band']
                                                  , axis = 1)
    t1 = time.time()
    total = t1 - t0

    # Print de doorlooptijd voor deze functie
    print('time ' + str(total))
    
    return df

print ('voor uitvoer van de functie: clean_data')
show(df_rates[df_rates['curve_type']=='Points'].head())

df_rates = clean_data(df_rates)

print ('na uitvoer van de functie: clean_data')
show(df_rates[df_rates['curve_type']=='Points'].head())

voor uitvoer van de functie: clean_data


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,12 MONTHS,NaT,1.0,-0.27084
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,12 MONTHS,NaT,10.0,-0.24084
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,12 MONTHS,NaT,11.0,-0.23584
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,12 MONTHS,NaT,2.0,-0.26984
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,12 MONTHS,NaT,3.0,-0.26884


time 2.0097594261169434
na uitvoer van de functie: clean_data


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,1 MONTH,2020-02-02,1.0,-0.27084
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,10 MONTHS,2020-11-02,10.0,-0.24084
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,11 MONTHS,2020-12-02,11.0,-0.23584
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,2 MONTHS,2020-03-02,2.0,-0.26984
2020-01-02,Swap 12M FRA,EUR,End of Day,Points,2020-01-02 15:07:48.277,3 MONTHS,2020-04-02,3.0,-0.26884


<a id='expand'></a>
# Expand Time Series

Nu gaan we op zoek naar de ontbrekende data in de rates.

Iedere curve kan uit een verschillend aantal datapunten bestaan. 
Per curve worden alle mogelijke Terms bekeken - op basis van de history - en wanneer dit datapunt niet wordt gevonden in de huidige data - dan wordt deze toegevoegd aan de lijst van exceptions.


In [5]:
def expand_time_series(df):
    """
    Vul de dataframe met alle mogelijke combinaties van keys voor elke rate_dt.
    
    Aan de dataframe wordt op basis van rate_dt en de sleutelvelden 
    alle mogelijke combinaties toegevoegd. Voor ontbrekende combinaties wordt 
    een NaN waarde ingevoegd.    

    Input: 
    df -              Pandas Datafame

    Output: 
    Pandas Dataframe      
    """           
    #Selectie van range van de laatste 20 waarnemingen
    dates = df['rate_dt'].unique()
    dates.sort()       
    
    # Doe de berekeningen alleen over de laatste N dagen.
    N = 20
    
    df_nd = df[df.rate_dt >= dates[-N]]
            
    keys =  ['curve_type','rate_name','rate_type','ccy','time_band'] 
    col = df.columns                
    
    # Alle combincaties van keys per laatste datum
    df_keys = df_nd[keys].drop_duplicates()  
    df_keys['rate_dt'] = df_nd['rate_dt'].max() 
        
    # mask = df_nd['rate_dt'] == df_nd['rate_dt'].max()                
                              
    df_expanded = df_keys.merge(df_nd, on=keys + ['rate_dt'], how='outer')        
            
    # kolommen herschikken
    df_expanded = df_expanded.reindex(columns=col)        
                              
    return df_expanded

print ('Before Expanding time series')
show(df_rates[(df_rates['rate_dt'] == df_rates['rate_dt'].max()) &
        (df_rates['rate_name'] == 'Deposit') &
        (df_rates['ccy'] == 'CHF')
       ]
    )

df_ext = expand_time_series(df_rates)

print ('Effect of expand_time_series')
show( df_ext[(df_ext['rate_dt'] == df_ext['rate_dt'].max()) &
             (df_ext['rate_name'] == 'Deposit') &
             (df_ext['ccy'] == 'CHF')
       ]
    )


Before Expanding time series


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,OVERNIGHT,2020-12-31,NaN,-0.88415
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,2 DAYS,2021-01-04,NaN,-0.96582
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,1 WEEK,2021-01-11,NaN,-0.86792
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,2 WEEKS,2021-01-18,NaN,-0.86707
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,1 MONTH,2021-02-04,NaN,-0.86176
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,3 MONTHS,2021-04-06,NaN,-0.83129
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,6 MONTHS,2021-07-05,NaN,-0.85090
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,9 MONTHS,2021-10-04,NaN,-0.82606
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,1 YEAR,2022-01-04,NaN,-0.80499


Effect of expand_time_series


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,OVERNIGHT,2020-12-31,NaN,-0.88415
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,2 DAYS,2021-01-04,NaN,-0.96582
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,1 WEEK,2021-01-11,NaN,-0.86792
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,2 WEEKS,2021-01-18,NaN,-0.86707
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,1 MONTH,2021-02-04,NaN,-0.86176
2020-12-30,Deposit,CHF,End of Day,Yield,NaT,2 MONTHS,NaT,NaN,NaN
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,3 MONTHS,2021-04-06,NaN,-0.83129
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,6 MONTHS,2021-07-05,NaN,-0.85090
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,9 MONTHS,2021-10-04,NaN,-0.82606
2020-12-30,Deposit,CHF,End of Day,Yield,2020-12-30,1 YEAR,2022-01-04,NaN,-0.80499


<a id='outliers'></a>
# Berekenen van outliers
  
Z-score wordt gedefinieerd over 20 dagen historie om outliers te herkennen
Daarnaast wordt bp_diff (Base Point Difference) berekend ten opzichte van de vorige dag.
Rate waar geen wijziging op heeft plaatsgevonden in de laatste 20 dagen worden gezien als outlier
Ook gaten in de rates worden toegevoegd als exceptions. De exceptions worden nu alleen berekend over de laatste dag. Een mooie uitbreiding zou kunnen zijn om ook exceptions over de voorliggende dagen te berekenen.

- $z$-score$ = \frac{x_i-\mu}{\sigma}  $

- $bp_{diff} = (rates_{(t)}.mid - rates_{(t-1)}.mid) * 100$
- No-change $= (rates_{(t)}.mid \overset{!}{=} rates.mid_{(t-1)} \overset{!}{=} rates.mid_{(t-n)}) $



In [6]:
# Bereken Bp diff tov vorige werkdag.
def bp_diff(x):
    """
    Bereken het verschil tussen de laatste en ena laatste waarde   
    Dit vereist dat de dataset gesorteerd is op datum.
    
    input:
        x   Pandas serries        
    """   
    x = np.asarray(x)    
    if len(x)>1:
        diff = (x[-1] -x[-2]) * 100
    else:
        diff = 0
    
    return diff 

# Bereken z-score
def zscore(x):
    """
    Bereken z-score, voor een array en geef alleen de laatste waarde terug
    input: 
        x   Pandas serries        
    """
        
    v = x.tail(1) 
    m = x.mean()
    s = x.std()        # Dit is de steekproef standaard deviatie
    
    z = (v - m ) / s
                    
    return z

def calculate_outliers(df):
    """               
    Bereken de std, bp_delta, en z-score
    Input:
        df     Pandas Dataframe. Dataframe should be a time-serries.
    Output
        df_gr  Pandas Dataframe        
    """      
    
    t0 = time.time()
    
    keys = ['curve_type', 'rate_name', 'rate_type', 'ccy', 'time_band']
    
    #Selectie van range van de laatste 20 waarnemingen
    dates = df['rate_dt'].unique()
    dates.sort()       
    
    # Bereken de Z-score over de laatste 20 werkdagen
    N = 20
    
    df_nd = df[df.rate_dt >= dates[-N]]
    df_nd = df_nd.sort_values(keys + ['rate_dt'])

    # Toevoegen van gemiddelde en standaard afwijking + berekenen van z-score   
    # with np.errstate wordt hier gebruikt om de waarschuwing voor devide by zero te onderdrukken
    with np.errstate(divide='ignore',invalid='ignore'):
        df_gr = df_nd.groupby(keys)['mid'].agg( 
            [('mean', np.mean),
             ('std', np.std), # Sample standard deviation
             ('zscore', zscore),
             ('bp_diff', bp_diff ),
             ('mid', lambda x: x.tail(1))
            ] ).reset_index()
        
    t1 = time.time()
    total = t1 - t0
    print('time ' + str(total))
    
    return df_gr
    
df_gr = calculate_outliers(df_ext)

print ('Berekenen gemiddelde, std, z-score, bp_diff')
show( df_gr[(df_gr['rate_name'] == 'Deposit') & (df_gr['ccy'] == 'CHF')])



time 1.7229387760162354
Berekenen gemiddelde, std, z-score, bp_diff


curve_type,rate_name,rate_type,ccy,time_band,mean,std,zscore,bp_diff,mid
Yield,Deposit,End of Day,CHF,1 MONTH,-0.835285,0.014879,-1.779393,-0.889,-0.86176
Yield,Deposit,End of Day,CHF,1 WEEK,-0.854320,0.037720,-0.360556,6.736,-0.86792
Yield,Deposit,End of Day,CHF,1 YEAR,-0.828453,0.009316,2.518594,0.486,-0.80499
Yield,Deposit,End of Day,CHF,2 DAYS,-0.861135,0.026245,-3.988707,-10.181,-0.96582
Yield,Deposit,End of Day,CHF,2 MONTHS,-0.813923,0.007505,NaN,NaN,NaN
Yield,Deposit,End of Day,CHF,2 WEEKS,-0.851908,0.024473,-0.619559,2.986,-0.86707
Yield,Deposit,End of Day,CHF,3 MONTHS,-0.813065,0.007018,-2.597030,-2.889,-0.83129
Yield,Deposit,End of Day,CHF,6 MONTHS,-0.859175,0.008390,0.986304,-1.014,-0.85090
Yield,Deposit,End of Day,CHF,9 MONTHS,-0.846710,0.009603,2.150371,0.986,-0.82606
Yield,Deposit,End of Day,CHF,OVERNIGHT,-0.844925,0.010112,-3.879132,-3.764,-0.88415


<a id='dimensions'></a>
# Toevoegen van dimensies

        Yield group en currency groups.        
        Dit wordt gedaan om beter te kunnen selecteren bij de presentatie van de rates
            

In [7]:
# Inlezen hiearchien

def add_hiarchy(df, fieldnames):
    """
    Deze functie voegt een dimensie toe die is gedefinieerd in de json data.
        
    Input: 
        df -          Pandas Datafame 
        fieldnames    List of names of hiarchies. 
                      In de data folder moet een bestand in JSON format staan met deze naam      
    Output: 
      Pandas Dataframe     
    """    
    try:

        for name in fieldnames:            
            # Lezen van hiarchie-en vanuit JSON
            filename = 'data\\' + name + ".json"
            df_r = pd.read_json(filename)
            df = pd.merge(df,df_r,how = 'left')
    
                
    except Exception as e:
        print("ERROR: Unable to find or access file:", e)
    
    return df

hiarchies = ['ccy_group','yield_group']

df_rates = add_hiarchy(df_rates,hiarchies)
df_gr  = add_hiarchy(df_gr,hiarchies)
df_ext = add_hiarchy(df_ext,hiarchies)

print ('Toevoegen ccy group en yield group')
show(df_rates.head())

Toevoegen ccy group en yield group


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid,ccy_group,yield_group
2020-01-01,European CPI,EUR,Fixing,Yield,2020-02-24,1 MONTH,2020-02-03,NaN,104.03246,A-currency,Bond Yield
2020-01-01,Netherlands CPI Base 2015,EUR,Fixing,Yield,2020-02-24,1 MONTH,2020-02-03,NaN,105.93230,A-currency,Bond Yield
2020-01-01,Spain CPI Base 2015,EUR,Fixing,Yield,2020-02-24,1 MONTH,2020-02-03,NaN,103.38178,A-currency,Bond Yield
2020-01-02,BBSW,AUD,Fixing,Yield,2020-01-02,1 MONTH,2020-02-03,NaN,0.84834,C-currency,Bond Yield
2020-01-02,BBSW,AUD,Fixing,Yield,2020-01-02,2 MONTHS,2020-03-02,NaN,0.87792,C-currency,Bond Yield


<a id='uitzonderingen'></a>
# Definieer de uitzonderingen 

1. Z-score te hoog of te laag (> 3.09) en Criteria op basis van basis punten
    * Bonds - 50 basis punten
    * Futures 50 basis punten
    * FX       2.5 basis punten
    * yield per tenor verschillend uit tabel
    
2. Punten die niet hebben bewogen in de laatste 20 dagen

3. Gaten in de data (Isna)


In [8]:
def define_treshold(df):
    """
    Threshold wordt bepaald om aan te geven wanneer een afwijking in basis punten te groot wordt bevonden
    
    Thresholds worden voor yields bepaald door combinaties van tenors en currencies uit de tabel gelezen
    Voor de overige curve types worden default tresholds gebruikt.
    
    Input: 
      df    Pandas Datafame       
    Output: 
      Pandas Dataframe     
    """
    
    # Mocht deze dataframe al een kolom treshold hebben verwijderen we deze eerst
    if 'threshold' in df.columns:            
        df = df.drop(columns = ['threshold'])            
    
    try:    
        # In deze tabel staan de Treshold waarden per ccy group, en tenor
        df_t = pd.read_excel(r'data\yield_tenor_treshold.xlsx')

        # Vertaal ccy groep in de treshold tabel naar dezelfde codering in de dataframe
        df_t.rename(columns = {'ccygroup':'ccy_group','timeband':'time_band'}, inplace = True)   
        
        # Map ccy group naar leesbare namen
        df_t['ccy_group'] = df_t['ccy_group'].map({1:'A-currency',2:'B-currency',3:'C-currency'})                
                    
        # Merge treshold tabel met de dataframe
        df = df.merge(df_t, how = 'left' )        
        
    except Exception as e:
        print("ERROR: Unable to find or access file:", e)
               
    # Set default values voor de overige tresholds
    df.loc[(df['curve_type']=='Capital Pr'),['threshold']] = 50
    df.loc[(df['curve_type']=='Futures'),['threshold']]    = 50
    df.loc[(df['curve_type']=='FX'),['threshold']]         = 2.5
    
    # Wanneer geen yield exceptie is opgegeven in de tabel - dan default 10.
    df.loc[((df['curve_type']=='Yield')),['threshold']] = df.loc[((df['curve_type']=='Yield')),['threshold']].fillna(10)
         
    return df


df_gr = define_treshold(df_gr)

print ('Toevoegen van tresholds')
show(df_gr.head())


Toevoegen van tresholds


curve_type,rate_name,rate_type,ccy,time_band,mean,std,zscore,bp_diff,mid,ccy_group,yield_group,threshold
Capital Pr,AT0000A001X2,End of Day,EUR,ACTUAL,102.98104,0.105877,-1.599970,-2.1,102.81164,A-currency,NaN,50.0
Capital Pr,AT0000A0U3T4,End of Day,EUR,ACTUAL,107.88436,0.114234,-1.484666,-3.1,107.71476,A-currency,NaN,50.0
Capital Pr,AT0000A105W3,End of Day,EUR,ACTUAL,107.01513,0.103418,-1.286525,-3.6,106.88208,A-currency,NaN,50.0
Capital Pr,AT0000A185T1,End of Day,EUR,ACTUAL,109.18143,0.115914,-1.165084,-5.7,109.04638,A-currency,NaN,50.0
Capital Pr,AT0000A1FAP5,End of Day,EUR,ACTUAL,109.26697,0.133820,-0.489089,0.1,109.20152,A-currency,NaN,50.0


In [9]:
# Voeg outliers samen...

def exception_type(row):    
    """ 
    Bepaal het type outlier
    
    Outside limit Exceptions worden bepaald adhv een limiet per rate, currency en tenor
    Stale rate Exceptions worden bepaald door te kijken naar de std deviatie over de laatste 20 dagen     
    Er is sprake van een Missing Data Exception indien er geen rates beschikbaar zijn voor dit Tenor punt 
    op de curve.
    Input:
        row   - rij van het dataframe
    Output:
        type exception string           
    """
    
    if (( ( row['zscore']> 3.09) | (row['zscore'] < -3.09) )
        & ( np.isfinite(row['zscore'] ) ) 
        & ( abs(row['bp_diff']) > row['threshold'] )):
        result = 'Outside Limits'
    elif row['std']==0:
        result = 'Stale Rate'
    elif np.isnan(row['mid']):
        result = 'Missing Data'
    else:
        result = ''
        
    return result
    
# Missing Data Exceptions toevoegen aan df_gr
df_exceptions = pd.merge(df_ext[df_ext['rate_dt']==df_ext['rate_dt'].max()], df_gr, how = 'outer')

# Voeg kolom toe met het soort exception
df_exceptions['exception'] = df_exceptions.apply(exception_type, axis=1)

# Verwijder regels die geen exception tonen
df_exceptions = df_exceptions[df_exceptions['exception'] != '']

 # Drop de missing data exceptions voor curve_types bonds en Futures
# Deze check is hiervoor niet zinvol
df_exceptions = df_exceptions[~((df_exceptions['curve_type']=='Futures') &
                              (df_exceptions['exception']=='Missing Data'))
                             ]
df_exceptions = df_exceptions[~((df_exceptions['curve_type']=='Capital Pr') &
                              (df_exceptions['exception']=='Missing Data'))
                             ]
            

print('Exceptions dataframe')
show(df_exceptions.head())

Exceptions dataframe


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid,ccy_group,yield_group,mean,std,zscore,bp_diff,threshold,exception
2020-12-30,CDS Curve DNB Bank Asa,EUR,End of Day,Yield,2020-12-30,6 MONTHS,2021-07-05,NaN,14.65536,A-currency,CDS Specific,14.65536,0.0,0.974679,0.0,10.0,Stale Rate
2020-12-30,CDS Curve DNB Bank Asa,EUR,End of Day,Yield,2020-12-30,1 YEAR,2022-01-04,NaN,15.76536,A-currency,CDS Specific,15.76536,0.0,0.974679,0.0,10.0,Stale Rate
2020-12-30,CDS Curve DNB Bank Asa,EUR,End of Day,Yield,2020-12-30,2 YEARS,2023-01-04,NaN,17.98536,A-currency,CDS Specific,17.98536,0.0,-0.974679,0.0,10.0,Stale Rate
2020-12-30,CDS Curve DNB Bank Asa,EUR,End of Day,Yield,2020-12-30,3 YEARS,2024-01-04,NaN,20.21536,A-currency,CDS Specific,20.21536,0.0,0.974679,0.0,10.0,Stale Rate
2020-12-30,CDS Curve DNB Bank Asa,EUR,End of Day,Yield,2020-12-30,4 YEARS,2025-01-06,NaN,21.10536,A-currency,CDS Specific,21.10536,0.0,-0.974679,0.0,10.0,Stale Rate


In [10]:
# Dit wordt gebruikt om onderdrukte verschillen te hiden. 
def filter_exceptions(df, df_filter):
    """
    Filters de geselecteerde velden van de 2e dataframe uit de eerste dataframe 
    
    Uit de eerste dataset worden de rijen verwijderd die overeenkomen met de 2e dataframe
    Input: 
      df                Pandas Datafame: dataset to be filtered
      df_filter         Pandas Datafame: dataset to be removed   
    Output: 
      Pandas Dataframe
    """   

    if len(df_filter) > 0:                 
        # Selecteer alleen rijen die wel voorkomen in df maar niet in df_filter
        dff = df.merge(df_filter, how = 'left', indicator=True)
        dff = dff.query('_merge == "left_only"').drop('_merge', 1)        
        
    else: dff = df
               
    return dff

In [11]:
# #Schrijf de data even naar Excel om verder te onderzoeken
# try:
#     df.to_excel(r'data\df.xlsx', index = False)
#     df_gr.to_excel(r'c:\temp\df_gr.xlsx', index = False)
#     df_exceptions.to_excel(r'data\df_exceptions.xlsx', index = False)

# except Exception as e:
#     print ("ERROR: Unable to find or access file:", e)

<a id='presentatie'></a>
# Presentatie van de resultaten

De data wordt gepresenteerd in de dashboard.

1. Overzicht van de 'Outliers': Outside limits, Stale Rates en Missing Data exceptions    
2. Tabblad voor FX Rates
3. Tabblad voor Interest Rates
4. Tabblad voor Bonds

Een aantal mooie voorbeelden van Python Plotly en Dash kan je hier vinden: https://dash-gallery.plotly.host/Portal/


In [12]:
# Hernoemen van curve_type waarde voor bonds voor de leesbaarheid in de presentatie
df_exceptions["curve_type"].replace({'Capital Pr': 'Bonds'}, inplace=True)
df_gr["curve_type"].replace({'Capital Pr': 'Bonds'}, inplace=True)
df_rates["curve_type"].replace({'Capital Pr': 'Bonds'}, inplace=True)


#Afronden op 5 decimalen voor presentatie
precision = 5
df_rates = df_rates.round(precision)
df_gr = df_gr.round(precision)
df_exceptions = df_exceptions.round(precision)

# Drop spot_end_of_day (duplicate data)
df_gr = df_gr[df_gr['rate_type']!='Spot End of Day']

# Prepare data for Tab1
header_text = """
In the data below you see the outliers defined by the business. 
A data point is considered an outlier if it meets one of the following criteria. 
Outside Limits exceptions are shown when a data point has moved outside the limits set by the Bank. 
We check on both the [z-score](https://en.wikipedia.org/wiki/Standard_score) > 3.09 and daily base point value movement. 
Stale Rates means the data point has not moved for at least 20 days. No Rate exceptions are shown when no rate is available for a datapoint in a curve on a tenor where we previously did have a data point. 
"""

subline = """
_Visual inspection of daily upload results and outliers._
""" + df_rates['rate_dt'].max().strftime('%d-%m-%Y')

# Kolommen die getoond worden bij de uitzonderingen
tab1_cols = ['exception','curve_type','rate_name','rate_type','ccy','time_band','mid','mean', 'std','zscore']
tab1_cols_alpha = ['exception','curve_type','rate_name','rate_type','ccy','time_band']

# Kolommen die getoond worden bij tab 2 
fx_cols = ['ccy_group', 'ccy', 'rate_type', 'mid', 'zscore', 'bp_diff']
fx_cols_alpha = ['ccy_group', 'ccy', 'rate_type']

# Kolommen die getoond worden bij tab 4 Bonds
bond_cols = ['rate_name', 'ccy', 'mid', 'zscore', 'bp_diff']
bond_cols_alpha = ['rate_name', 'ccy']

# Prepare data for Tab 3: interest Rates and FRA Points
df_ir = df_rates.loc[df_rates['curve_type'].isin(['Yield','Points']) ]

# drop alle data punten langer dan 30 jaar (alleen voor presentatie)
df_ir = df_ir[~df_ir['time_band'].isin(['35 YEARS','40 YEARS','45 YEARS','50 YEARS','55 YEARS','60 YEARS'])]
df_ir = df_ir.sort_values(['rate_name','rate_dt','actual_dt']).reset_index(drop=True)

# Drop yield groups Recovery Rates en Bond Yields - Geen curve data
df_ir = df_ir[~df_ir['yield_group'].isin(['Recovery Rate','Bond Yield'])]
        
# Sorteren van data voor line plots en datatable exceptions
keys = ['rate_dt','curve_type','rate_name','rate_type','ccy','time_band']
df_exceptions.sort_values(by=keys, inplace=True)
df_rates.sort_values(by=keys, inplace=True)

# Sorteren van data voor datatables fx en bonds
keys = ['curve_type','rate_name','rate_type','ccy','time_band']
df_gr.sort_values(by=keys, inplace=True)




In [13]:
def selected_to_df(tablerows, selected_rows):
    """
    Vertaal selectie uit Dash datatable naar een pandas dataset
    
    Input:
    tablerows:     dict: data vanuit datatable object
    selected_rows: list: list van indexes van selected rows van data table                   
    
    Output:
    Pandas dataframe
    """
          
    if len(tablerows) > 0:
        
        cols = list(tablerows[0].keys())        
        
        # New dataframe
        dfi = pd.DataFrame(cols)  

        for i in selected_rows:
            row = tablerows[i]  
            rowkeys = { k: row[k] for k in cols}
            dfi = dfi.append(rowkeys,ignore_index=True)  
            
    else:
        # return een lege dataframe
        dfi = pd.DataFrame()  
                
    
    return dfi

In [14]:
# Translate ccy to ccy pair based on dominant ccy
def ccypair(ccy):
    """
    Translate ccy to ccy pair based on dominant ccy with base ccy USD
    
    Input: 
        ccy     String
    Output:
        ccypair String
    
    """
    if ccy in ['EUR','GBP','AUD','NZD']:
        return ccy + '/USD'
    else:
        return 'USD/'+ ccy     
    

In [15]:

def linechart(df,x,y, title, xaxis_title, yaxis_title,color=None, hover_mode=False, hover_column=None):
    """
    Construct line graph
    
    Input: 
    df:           dataframe
    x:            x-axis
    y:            y-axis   
    title:        chart title
    xaxis-title   title xaxis
    xaxis-title   title yaxis
    color:        group by variable - default = None
    hover_mode    plotly hover value (x,y,closest,x unified) default is False
    hover_column  Optioneel: kolom die terug gegeven moet worden als custom data

    output:
    fig           dict

    """
    if len(df.index) == 0:           
        fig = {}
    else:        
        if hover_column is not None:
            fig = px.line( df, x=x, y=y, color = color,
                       custom_data=[hover_column]  )
        else:
            fig = px.line( df, x=x, y=y, color = color)


        fig.update_traces(mode="lines", connectgaps=True)                       

        fig.update_layout( title=title, xaxis_title=xaxis_title, yaxis_title= yaxis_title,hovermode=hover_mode,
                       xaxis_tickformat = '%d-%m-%Y' )
                
        
        
        # Custom Hover tips - helaas kon ik de koptekst niet aanpassen
        if hover_column is not None:        
            fig.update_traces(
                hovertemplate="<br>".join([            
                    "%{y}",
                    "%{customdata[0]}"        
                ])
        )        
    return fig

In [19]:
# Build Dash App

# Html Css Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Build App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Dit is noodzakelijk om callsback van de tabs die nog niet gerendered zijn, te kunnen definieren
app.config['suppress_callback_exceptions'] = True

# Main application layout
app.layout = html.Div(
    [       
        html.Div( 
            [
                 html.Div( children = 
                     [
                        html.H1( "Check Marktdata " ),                  
                        dcc.Markdown(subline),     
                     ]
                     , style={'display': 'inline-block', 'width': '49%'}
                 ),
                 html.Div(                      
                     html.Img(src=app.get_asset_url('VFM_logo.gif'), style={ 'width' : 400, 'float' : 'right' } )                 
                 , style={'display': 'inline-block', 'width': '49%'}
                 )
            ]
        ),                 
        html.Br(),  
        dcc.Tabs(
            id='tabs-overview', 
            value='tab-1', 
            children=[            
                dcc.Tab(label='Exceptions', value='tab-1'),
                dcc.Tab(label='FX', value='tab-2'),
                dcc.Tab(label='Interest Yield', value='tab-3'),                
                dcc.Tab(label='Bonds', value='tab-4')                
            ]
        ),
        html.Div(
            id='tabs-overview-content'
        )
    ], style= { 'width': '100%', 'display': 'inline-block', 'padding-left':'2%', 'padding-right':'2%' }    
)
        
@app.callback(Output('tabs-overview-content', 'children'),
              Input('tabs-overview', 'value'))
def render_content(tab):
    """
    Callback voor de main applicatie.
    
    Callback functies worden gebruikt in Dash om schermobjecten 
    te koppelen via Dash input en output parameters. De ID's van de input en output komen terug in de layout.
    De input variabelen moeten overeenkomen met de dash Input waarde in de callback functie.
    in dit geval wordt de waarde van de id tabs-overview doorgegeven in de variable tab.
    
    Input:
        dcc.tab, value
    Output:
        dcc.tab, tabcontent
    """    
    if tab == 'tab-1':         
        return tab1             
    elif tab == 'tab-2':
        return tab2        
    elif tab == 'tab-3':
        return tab3            
    elif tab == 'tab-4':
        return tab4               

# Render Tab1 : Exceptions data

tab1 = html.Div(
    [
        html.Div( 
            dcc.ConfirmDialog( 
                id='tab1_confirm',
                message='Are you sure you want to supress warnings for these rates?'
            )
        ),        
        html.Div(
            [   
                html.H3(
                    id = 'tab1_header',    
                    children=["Exceptions"]
                ), 
                dcc.Markdown(header_text),  
                html.Br(),
                dcc.Checklist(
                    id = 'tab1_checklist_exceptions', 
                    options=[{'label': i, 'value': i} for i in df_exceptions['exception'].unique()], 
                    labelStyle={'display': 'inline-block'},         
                ),            
                html.Div(
                    [ 
                        dct.DataTable( 
                            id='tab1_datatable_exceptions',                             
                            row_selectable='multi', 
                            page_size=20,                             
                            # Left allign de tekst kolommen                        
                            style_cell_conditional = [ { 'if': {'column_id': c}, 'textAlign': 'left' } for c in tab1_cols_alpha ],
                            # gestreepte regels voor de leesbaarheid
                            style_data_conditional=[ { 'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)' }],
                            # achtergrond kopregel
                            style_header={'backgroundColor': 'rgb(230, 230, 230)','fontWeight': 'bold' },                                            
                            # Style as list
                            style_as_list_view=True
                        )               
                    ]
                )                    
            ], style={'display': 'inline-block', 'width': '70%','padding-left':'2%', 'padding-right':'2%'}
        ),
        html.Div(
            [    
                dcc.RadioItems(
                    id='tab1_radio_show_suppressed', 
                    options= [ {'label': 'Suppressed', 'value': 1}, {'label':'Exceptions', 'value': 0}],
                    value=0,            
                    labelStyle={'display': 'inline-block'}
                ),                
                html.Br(),
                html.P( 'Hide or show the exceptions '),
                html.Button(
                    'Hide', 
                    id='tab1_hide_button', 
                    n_clicks=0
                ),            
            ], style={'display': 'inline-block','padding-left':'2%', 'padding-right':'2%'}               
        )
    ]
)

@app.callback(
    Output('tab1_header','children'),
    Input('tab1_radio_show_suppressed','value')
)
def update_radio(show_suppressed):
    """
    Callback tab1 radio box
    
    Update de header text afhankelijk van de waarde van de radio box    
    Input:   
        dcc.radio.value
    Output:
        html.H3.value string
    """   
    if show_suppressed:
        return 'Suppressed'
    else:
        return 'Exceptions'


@app.callback(    
    Output('tab1_confirm','message'),
    Input('tab1_radio_show_suppressed','value')
)
def update_confirmdialog(show_suppressed):
    """    
    Callback tab1 confirmation dialog
    
    Wijzig de tekst van de confirmatie dialoog afhankelijk van de radio box
    Input: 
        dcc.radio.value
    Output:
        dcc.confirm.message string        
    """    
    if show_suppressed:
        return 'unhide the selected exceptions?'
    else:
        return 'Are you sure you want to supress warnings for these rates?'
            
   
@app.callback(
    Output('tab1_hide_button','children'),
    Input('tab1_radio_show_suppressed','value')        
)
def update_button(show_suppressed): 
    """    
    Callback tab1 button
    
    Wijzig de naam van de button afhankelijk van de radio box.
    Input: 
        dcc.radio.value
    Output:
        dcc.button.text string            
    """        
    if show_suppressed:
        return 'Unhide'
    else:
        return 'Hide'
    
@app.callback(    
    Output('tab1_confirm', 'displayed'),
    Input('tab1_hide_button', 'n_clicks'),        
    State('tab1_datatable_exceptions', 'selected_rows')
)
def update_out(n_clicks, selected):
    """    
    Callback tab1 confirmation dialog
    
    Toon de dialog als er op de button geklikt is
    en er iets in de tabel is geselecteerd.             
    """                
    if selected: 
        return True
    else:
        return False
    
@app.callback(Output('tab1_datatable_exceptions', 'data'),
              Output('tab1_datatable_exceptions', 'columns'),
              Output('tab1_datatable_exceptions', 'selected_rows'),
              Input('tab1_checklist_exceptions', 'value'),
              Input('tab1_confirm', 'submit_n_clicks'),
              Input('tab1_radio_show_suppressed','value'),           
              State('tab1_datatable_exceptions', "data"),
              State('tab1_datatable_exceptions', "selected_rows")              
             )
def show_data(checklist_exceptions, savebutton, show_suppressed, tablerows, selected_rows):                        
    """
    Toon in datatable de resultaten afhankelijk van de de input
    
    Input:
        dcc.checklist.value         -> gekozen type exceptions
        dcc.confirm.submit_n_clicks -> resultaat van de confirm dialoog (aanpassingen opslaan)
        dcc.radiobox                -> tonen onderdrukte excepties of niet
    State:
        dcc.datatable.tablerows     -> list met waarden in de tabel
        dcc.datatable.selected_rows -> selectie in de datatable        
    Output:
        dcc.datatable.data          -> te tonen data
        dcc.datatable.columns       -> te tonen kolomen
        dcc.datatable.selected_rows -> te selecteren rijen in de datatable        
    """    
    if selected_rows is None:
        selected_rows = []
           
    # determine context - dwz welke input voorzaakte de call?
    ctx = callback_context
              
    if ctx.triggered:            
        # Component dat de callback aanroept
        input_component = ctx.triggered[0]['prop_id'].split('.')[0]
                    
        if input_component == 'tab1_confirm':
            
            if not selected_rows:
                print ('ERROR: Selection made with 0 items in callback tab1')
            else:                    
                if show_suppressed:

                    # de geselecteerde regels moeten worden verwijderd uit de exception lijst                                              
                    df_hide_exceptions = pd.DataFrame.from_dict(tablerows)                     
                    df_deleted = selected_to_df(tablerows, selected_rows)                    
                    df_hide_exceptions = filter_exceptions(df_hide_exceptions,df_deleted)

                else:                        
                    try:
                        df_hide_exceptions = pd.read_excel(r'data\exceptions.xlsx')
                                                                
                        # Voeg de geselecteerde excepties toe aan de lijst van uitzonderingen
                        for i in selected_rows:
                            row = tablerows[i]
                            rowkeys = { k: row[k] for k in ['exception','curve_type','rate_name','rate_type','ccy','time_band'] }
                            df_hide_exceptions = df_hide_exceptions.append(rowkeys,ignore_index=True)   

                    except Exception as e:
                        print("ERROR: Unable to find or access file:", e)                    
            
                try:
                    # save the hide exceptions dataframe - to disc                
                    df_hide_exceptions.to_excel(r'data\exceptions.xlsx', index = False)
                    
                    # Deselect de datatable rows
                    selected_rows = []

                except Exception as e:
                    print("ERROR: Unable to find or access file:", e)                        

    try:
        
        # lees de te excepties in
        df_hide_exceptions = pd.read_excel(r'data\exceptions.xlsx') 
                        
            
        # toon de onderdrukte verborgen excepties
        if show_suppressed:      
            
            dff_hide_exceptions = df_hide_exceptions
            
            # Filter de geselecteerde exceptions
            if checklist_exceptions:
                dff_hide_exceptions = df_hide_exceptions[df_hide_exceptions['exception'].isin(checklist_exceptions)]
            
            # Vertaal de dataframe naar dict met data voor datatable object
            if len(dff_hide_exceptions.index) > 0:
                data = dff_hide_exceptions[tab1_cols_alpha].to_dict('records')
            else:
                data = []
            
            # Toon de kolomen die behoren bij deze datatable
            columns = [{"name": i, "id": i} for i in tab1_cols_alpha]                      
           
        # toon de afwijkingen in de data
        else:
              
            # Check of een of meer type excepties getoont moeten worden, en beperk zo nodig de selectie
            dff_exceptions = df_exceptions
            
            # Filter de geselecteerde exceptions
            if checklist_exceptions:
                dff_exceptions = df_exceptions[df_exceptions['exception'].isin(checklist_exceptions)]

            # Filter de uitzonderingen uit de lijst die we niet willen zien
            dff_exceptions = filter_exceptions(dff_exceptions, df_hide_exceptions)

            # Vertaal de dataframe naar dict met data voor datatable object
            data = dff_exceptions[tab1_cols].to_dict('records')
            
             # Toon de kolomen die behoren bij deze datatable
            columns = [{"name": i, "id": i} for i in tab1_cols]            
        
    except Exception as e:
        print("ERROR: Unable to find or access file:", e)
        
    return data, columns, selected_rows

# Tab2 FX resultaten     
tab2 = html.Div(
            [   html.H3('FX'),
                html.P('Show Daily FX Rates'),  
                html.Div (  
                    [
                        html.Div (
                            [ 
                                dct.DataTable( 
                                    id='tab2_datatable', 
                                    columns = [{"name": i, "id": i} for i in fx_cols], 
                                    data= df_gr.loc[(df_gr['curve_type']=='FX')][fx_cols].to_dict('records'),
                                    row_selectable='single',   
                                    selected_rows = [0],
                                    page_size=20,                                     
                                    style_cell_conditional = [ { 'if': {'column_id': c}, 'textAlign': 'left' } for c in fx_cols_alpha] ,
                                    # gestreepte regels voor de leesbaarheid                                
                                    style_data_conditional=[ { 'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)' }],                              
                                    # achtergrond kopregel
                                    style_header={'backgroundColor': 'rgb(230, 230, 230)','fontWeight': 'bold' },                                            
                                    style_as_list_view=True,
                                    sort_action="native",
                                    sort_mode="single"                                     
                                )
                            ], 
                            style={'display': 'inline-block','width': '49%','height': '10px' }
                        ),                        
                        html.Div( [ 
                                    dcc.Checklist(id='tab2_chkbox',
                                        options = [
                                            {'label': 'Show ECB Standard Close Rates', 'value': 'ECB'} 
                                        ]
                                    ),
                                    dcc.Graph(id="tab2_line_chart")
                                  ],
                                  style = {'display': 'inline-block','width': '49%','height': '10px'}
                                )                                   
                    ]
                )  
            ], 
    style={'display': 'inline-block', 'width': '85%','padding-left':'2%', 'padding-right':'2%'}            
    )

@app.callback(
     Output("tab2_line_chart", "figure"), 
     Input("tab2_datatable", "data"),
     Input("tab2_datatable", "selected_rows"), 
     Input('tab2_chkbox','value')
)
def update_line_chart(data, selected_row, ecb):
    """
    Call back van de lijn plot voor FX
    
    Toon de lijnplot afhankelijk van de selectie in de data tabel 
    en de checkbox keuze voor ECB rates.
    
    Input:
       dcc.datatable.data             Data table data
       dcc.datatable.selected_rows    Selectie in de datatable
       dcc.checkbox.value             Keuze om ECB rates te tonen
    Output:
       dcc.Graph                      lijnplot
    """
    
    
    if selected_row:
        ccy = data[selected_row[0]]['ccy']    
        
        # selecteer de juiste FX rates
        df_fx_hist = df_rates.loc[(df_rates['curve_type']=='FX') & (df_rates['rate_type']!='Spot End of Day')]        
        mask = df_fx_hist['ccy']==ccy                        
        
        # maak een plotly graph aan met de lijn plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_fx_hist[mask]['rate_dt'], y= df_fx_hist[mask]['mid'],
                            mode='lines',                            
                            name='Mid Close Rates'))

        # voeg evt. de ECB Close rates toe indien gewenst
        if ecb:
            mask = df_ecb['ccy']==ccy 
            fig.add_trace(go.Scatter(x=df_ecb[mask]['rate_dt'], y= df_ecb[mask]['usdrate'],
                            mode='lines',
                            marker=dict(
                                color='pink'),
                            name='ECB Rates'))            
        
        # Update titles
        title = 'Daily FX Rates ' + ccypair(ccy)               
        fig.update_layout(
            title=title,
            xaxis_title="Rate Date",
            yaxis_title="Daily MID Close Rate",    
        )
        
        # toon de legenda
        fig.update_layout(showlegend=True)
        
        # toon de lijn plot
        return fig
    else:
        return {}

    
# Tab3: Interest Rates
tab3 = html.Div(
    [
        html.H3('Interest Rates'),
        html.P('Show Daily PAR Rates and history'),  
        html.Div(
            [
                html.Div(
                    [
                        dcc.Dropdown(
                            id='tab3_ddlist_yieldgroup',
                            options=[{'label': i, 'value': i} for i in df_ir['yield_group'].unique()],
                            value = 'Swap Rates (Bond Based)', #df_ir['yield_group'].iloc[-1],
                            clearable=False
                        )            
                    ], style={'width': '24%', 'display': 'inline-block'}),
                html.Div(
                    [
                       dcc.Dropdown(
                            id='tab3_ddlist_ccy',                            
                            clearable=False
                        )              
                    ], style={'width': '24%', 'display': 'inline-block'})            
            ]
        )
        ,        
        html.Div(id='tab3-output-container', 
                 children =
                    [
                        dcc.Graph(id='tab3-graph'),
                        html.Div(id = 'tab3-hist_containter',
                                 children = 
                                     [                                        
                                        dcc.Graph(id='tab3-graph-hist')            
                                     ]
                        )
                    ]
        )        
    ]
)


@app.callback(
    Output('tab3-hist_containter','style'),
    Input ('tab3-graph','hoverData')  
)
def hide_block(hover_data):
    ''' Hide history graph als er niets is geselecteerd. '''    
    if hover_data is None or hover_data == []:
        return {'display': 'none'}
    else:
        return {'display': 'block'}
        
@app.callback(
    Output('tab3_ddlist_ccy','options'),
    Output('tab3_ddlist_ccy','value'),
    Input('tab3_ddlist_yieldgroup','value'),
    State('tab3_ddlist_ccy','value')
)
def select_ccy(yield_group, ccy):    
    """
    Toon de lijst met currencies in de ccy dropdown 
    
    De waarden in de ccy drop down waarden wordt bepaald adhv de 
    keuze in de yieldgroep drop down list    
    Input:
        yieldgroup dcc.dropdown_list.value  (huidige selectie van yieldgroup)
        ccy dcc.dropdown_list.value (huidige selectie van ccy)
    Output:
        ccy dcc.dropdown_list.options (lijst met keuze in de ccy dropdown)
        value dcc.dropdown_list.value (Geselecteerde keuze in de ccy dropdown)               
    """         
            
    # Selecteer meest actuele datum
    dff_ir = df_ir[df_ir['rate_dt']==df_ir['rate_dt'].max()]    
    
    # Drop down selectie van Yield Group
    if yield_group is not None:
        dff_ir = dff_ir[dff_ir['yield_group'] == yield_group]    
              
    # Indien Rates niet aanwezig zijn of nog geen ccy gekozen is selectie wijzigen naar EUR.
    if ccy is None:
        ccy = 'EUR'
    
    # Check of de gekozen ccy voorkomt in de lijst
    if len (dff_ir.query('ccy == "' + ccy + '"')) == 0:
        # Zo niet, dan wordt de keuze EUR.
        # Wanneer EUR ook niet voorkomt, nemen we de eerste ccy in de lijst
        if len (dff_ir.query('ccy =="EUR"')) != 0:            
            ccy = 'EUR'
        else:        
            ccy = dff_ir['ccy'].iloc[0]          
        
    # Update selectie lijst en gekozen ccy   
    options_list = [{'label': i, 'value': i} for i in dff_ir['ccy'].unique()]
            
    return options_list , ccy
         
@app.callback(
    Output('tab3-graph', 'figure'),
    Input ('tab3_ddlist_yieldgroup','value'),
    Input ('tab3_ddlist_ccy','value')    
)
def display_graph(yield_group, ccy):
    """
    Toon de curve in een lijnplot afhankelijk van de gemaakte selecties
    
    Input:
        yield_group    dcc.dropdownlist.value    geselecteerde yield_group
        ccy dcc.dropdownlist.value  geselecteerde ccy
    Output:
        dcc.graph lijnplot van de geselecteerde curve.    
    """
                    
    if ccy is not None and yield_group is not None:
                
        # Selecteer meest actuele datum
        dff_ir = df_ir[df_ir['rate_dt']==df_ir['rate_dt'].max()]        

        # Drop down selectie van Yield Group en Ccy
        dff_ir = dff_ir[dff_ir['yield_group'] == yield_group]    
        dff_ir = dff_ir[dff_ir['ccy'] == ccy]
        
        # Toon alleen de grafiek als de selectie niet leeg is
        if len(dff_ir.index)== 0:
            fig = {}
        else:
            # Graph Title
            title = 'Interest Yield ' + ccy + ' ' + yield_group
            fig = linechart(dff_ir,
                            'actual_dt',
                            'mid',
                            title,
                            "Actual Date",
                            "Daily Mid Close Rate", 
                            'rate_name', 
                            'x unified',
                            'time_band')
            fig.update_traces(mode="lines+markers", connectgaps=True)        
    else:
        fig = {}
        
    return fig



@app.callback(
    Output('tab3-graph-hist', 'figure'),
    Input ('tab3_ddlist_yieldgroup','value'),
    Input ('tab3_ddlist_ccy','value'),
    Input ('tab3-graph','hoverData'),
    State ('tab3-graph','figure')
)
def display_hist(yield_group, ccy, hover_data, figure):
    """ 
    Toon de historisch grafiek afhankelijk van het datapunt van de hoverwaarde in de dagelijkse grafiek   
    
    Input:
        yield_group    dcc.dropdownlist.value    geselecteerde yield_group
        ccy            dcc.dropdownlist.value    geselecteerde ccy
        hover_data     dcc.graph.hoverData       geselecteerd datapunt
    State:
        figure         dcc.graph.figure          dagelijkse grafiek
    Output:
        figure         dcc.graph.figure          historische grafiek    
    """
    
                    
    if hover_data and ccy is not None and yield_group is not None and figure is not None: 
        
        dff_ir = df_ir
        
        # Time Band is needed to select the historical rates for data point x
        time_band = hover_data['points'][0]['customdata'][0]
                      
        if time_band:
            # Drop down selectie van Yield Group en Ccy
            dff_ir = dff_ir[dff_ir['yield_group'] == yield_group]    
            dff_ir = dff_ir[dff_ir['ccy'] == ccy]        
            dff_ir = dff_ir[dff_ir['time_band'] == time_band]
        
            title = 'Interest Yield history ' + ccy + ' ' + yield_group + ' ' + time_band 
            title += '<br>' + 'Historical view showing the movement of one data point in time.'
        
            if len(dff_ir.index)== 0:
                fig = {}
            else:      
                fig = linechart(dff_ir,'rate_dt','mid', title,'Rate Date','Daily Mid Close Rate','rate_name', 'closest' )
                        
                       
            # Probleem met de kleuren: deze komen niet altijd overeen met de oorspronkelijke chart wat nogal verwarrend is
            # kleuren van de historische grafiek afstemmen op de dagelijkse grafiek
            colordict = {}
            # Get colors van main graph
            if 'data' in figure:
                for i in figure['data']:
                    if 'legendgroup' in i:
                        rate_name = i['legendgroup']
                        if 'line' in i:
                            if 'color' in i['line']:
                                color = i['line']['color']                                
                                colordict[rate_name] = color           
            # Set colors based on the main graph
            if 'data' in fig:
                for i in fig['data']:
                    if 'legendgroup' in i:
                        rate_name = i['legendgroup']
                        if 'line' in i:
                            if 'color' in i['line']:
                                if rate_name in colordict:
                                    i['line']['color'] = colordict[rate_name]
                                                              
        else:
            fig = {}                   
    else:
        fig =  {}        
    return fig


# Tab4 Bonds
tab4 = html.Div(
            [   html.H3('Bond Prices'),
                html.P('Show Daily Bond Prices'),  
                html.Div (  
                    [
                        html.Div (
                            [ 
                                dct.DataTable( 
                                    id='tab4_datatable', 
                                    columns = [{"name": i, "id": i} for i in bond_cols], 
                                    data= df_gr.loc[(df_gr['curve_type']=='Bonds')][bond_cols].to_dict('records'),
                                    row_selectable='single',   
                                    # selecteer eerste regel
                                    selected_rows = [0], 
                                    page_size=20,                                     
                                    style_cell_conditional = [ { 'if': {'column_id': c}, 'textAlign': 'left' } for c in bond_cols_alpha],
                                    # gestreepte regels voor de leesbaarheid                                
                                    style_data_conditional=[ { 'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)' }],
                                    # achtergrond kopregel
                                    style_header={'backgroundColor': 'rgb(230, 230, 230)','fontWeight': 'bold' },
                                    style_as_list_view=True,
                                    sort_action="native",
                                    sort_mode="single"                                    
                                )
                            ], 
                            style={'display': 'inline-block','width': '49%','height': '10px' }
                        ),
                        html.Div( dcc.Graph(id="tab4_line_chart"), 
                                  style = {'display': 'inline-block','width': '40%','height': '10px'}
                                )                                   
                    ]
                )  
            ], 
    style={'display': 'inline-block', 'width': '85%','padding-left':'2%', 'padding-right':'2%'}            
    )


@app.callback(
     Output("tab4_line_chart", "figure"), 
     Input("tab4_datatable", "data"),
     Input("tab4_datatable", "selected_rows"),    
)
def update_bond_chart(data, selected_row):
    """
    Toon Bond grafiek afhankelijk van de keuze in de datatable
    
    In deze datatable kan maar maximaal 1 regel geselecteerd worden. 
    De bond grafiek toont de grafiek van de geselecteerde regel over de tijd.
    
    Input:
        dcc.datatable.data           data van de datatable
        dcc.datatable.selected_rows  gemaakte keuze in de datatable
    Ouput:
        dcc.graph.figure             line chart van de geselecteerde bond
    """    
            
    # indien er iets is geselecteerd
    if selected_row:
        
        # filter de historische bond rates
        df_bonds_hist = df_rates.loc[(df_rates['curve_type']=='Bonds') ]        
        
        # naam van de bond die we willen zien        
        bondname = data[selected_row[0]]['rate_name']                
        
        # selectie binnen de bonds
        mask = df_bonds_hist['rate_name']==bondname 
        
        # titel van de plot
        title = 'Daily Bond Prices for ISIN ' + bondname
        
        # maak de plot aan
        fig = linechart(df_bonds_hist[mask],
                        'rate_dt',
                        'mid',
                        title,
                        "Rate Date",
                        "Daily Mid Close Rate",
                        None,
                        'closest')
        return fig
    else:
        return {}

# Run the App
app.run_server()

Dash app running on http://127.0.0.1:8050/
